## Install Required Libraries
These should already be installed if using the ServiceWorkbench product.

In [ ]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("DESeq2")
#BiocManager::install("tximport")
#BiocManager::install("tidyverse")
#BiocManager::install("botor")
#BiocManager::install("apeglm")
#BiocManager::install("ashr")
#BiocManager::install("vsn")
#BiocManager::install("pheatmap")
#BiocManager::install("tximportData")

In [ ]:
library("tximportData")
library("DESeq2")
library("tximport")
library("tidyverse")
library("botor")
library("apeglm")
library("ashr")
library("readr")

# Import Data
At a high level, we need a named vector of file paths (S3 URIs). These will be read in directly from S3. We also need the correct tx2gene mappings which can be retrieved from the tximportData library as shown below. The transcripts present in the analysis should match the mapping file used.

## Option 1: Import your own data from S3

In [ ]:
importer_function <- function(x) {
    s3_read(x, read_tsv)
}

In [ ]:
# Read in samples description file, expected to have a column which describes the condition to test against and rownames which map to the samples named in the file_paths vector below
sample_names <- c("ERR188297","ERR188088","ERR188329","ERR188288","ERR188021","ERR188356")
samples <- data.frame("run" = sample_names)
samples$condition <- factor(rep(c("A","B"), each=3))
rownames(samples) <- samples$run
samples

In [ ]:
file_paths <- c(
    "s3://<BUCKET>ERR188297.isoforms.results.gz",
    "s3://<BUCKET>ERR188088.isoforms.results.gz",
    "s3://<BUCKET>ERR188329.isoforms.results.gz",
    "s3://<BUCKET>ERR188288.isoforms.results.gz",
    "s3://<BUCKET>ERR188021.isoforms.results.gz",
    "s3://<BUCKET>ERR188356.isoforms.results.gz")
names(file_paths) <- samples$run
file_paths

In [ ]:
# Read in the tx2gene csv
gencode <- read_csv(file.path(dir,"tx2gene.gencode.v27.csv"))
ensembl <- read_csv(file.path(dir,"tx2gene.ensembl.v87.csv"))
refseq <- read_csv(file.path(dir,"tx2gene.csv"))
tx2gene <- gencode

In [ ]:
txi <- suppressMessages(tximport(
  files = file_paths,
  type = "rsem",
  txIn = TRUE,
  txOut = TRUE,
  importer = importer_function,
  tx2gene=tx2gene,
  existenceOptional = TRUE))

## Option 2: Grab Example Data from tximportData

In [ ]:
dir <- system.file("extdata", package = "tximportData")
samples <- read.table(file.path(dir, "samples.txt"), header = TRUE)
samples$condition <- factor(rep(c("A","B"),each=3))
file_paths <- sort(list.files(file.path(dir, "rsem"), pattern="*isoforms.results.gz", full.names=TRUE, recursive=TRUE))
names(file_paths) <- sort(samples$run)
file_paths

In [ ]:
samples

In [ ]:
# Read in the tx2gene csv
gencode <- read_csv(file.path(dir,"tx2gene.gencode.v27.csv"))
ensembl <- read_csv(file.path(dir,"tx2gene.ensembl.v87.csv"))
refseq <- read_csv(file.path(dir,"tx2gene.csv"))
tx2gene <- gencode

In [ ]:
txi <- suppressMessages(tximport(
  files = file_paths,
  type = "rsem",
  txIn = TRUE,
  txOut = TRUE,
  tx2gene=tx2gene,
  existenceOptional = TRUE))

In [ ]:
head(txi$counts)

# DESeq2 Analysis

In [ ]:
# Read in count matrix created using tximport
ddsTxi <- DESeqDataSetFromTximport(txi, colData = samples, design = ~ condition)

In [ ]:
dds <- DESeq(ddsTxi)

In [ ]:
res <- results(dds)
res

In [ ]:
resultsNames(dds)

## Log fold change shrinkage for visualization and ranking

*Note:* The coefficient name is dependent on the condition configured as part of the experiment. Select the correct name from the output of the previous command (`resultsNames(dds)`)

In [ ]:
coef_name <- "condition_B_vs_A"
resLFC <- lfcShrink(dds, coef=2, type="apeglm")

In [ ]:
resLFC

## p-values and adjusted p-values

In [ ]:
resOrdered <- res[order(res$pvalue),]

In [ ]:
# Summarize basic tallies
summary(res)

In [ ]:
# How many adjusted p-values are less than 0.1?
sum(res$padj < 0.1, na.rm=TRUE)

## Exploring and exporting Results

### MA-plot

In [ ]:
plotMA(res, ylim=c(-2,2))

In [ ]:
plotMA(resLFC, ylim=c(-5,5))

## Alternative shrinkage estimators

In [ ]:
resNorm <- lfcShrink(dds, coef=2, type="normal")
resAsh <- lfcShrink(dds, coef=2, type="ashr")

In [ ]:
par(mfrow=c(1,3), mar=c(4,4,2,1))
xlim <- c(1,1e5); ylim <- c(-3,3)
plotMA(resLFC, xlim=xlim, ylim=ylim, main="apeglm")
plotMA(resNorm, xlim=xlim, ylim=ylim, main="normal")
plotMA(resAsh, xlim=xlim, ylim=ylim, main="ashr")

## Plot Counts

In [ ]:
plotCounts(dds, gene=which.min(res$padj), intgroup="condition")

In [ ]:
d <- plotCounts(dds, gene=which.min(res$padj), intgroup="condition", 
                returnData=TRUE)
library("ggplot2")
ggplot(d, aes(x=condition, y=count)) + 
  geom_point(position=position_jitter(w=0.1,h=0)) + 
  scale_y_log10(breaks=c(25,100,400))

In [ ]:
mcols(res)$description

## Exporting results to CSV files

In [ ]:
write.csv(as.data.frame(resOrdered),
         file="condition_treated_results.csv")

In [ ]:
#Exporting only the results which pass an adjusted p value threshold can be accomplished with the subset function, followed by the write.csv function.
resSig <- subset(resOrdered, padj < 0.1)
resSig

# Data transoformations and visualizations

## Count data transformations

## Extracting transformed values

In [ ]:
vsd <- vst(dds, blind=FALSE)
rld <- rlog(dds, blind=FALSE)
head(assay(vsd), 3)

## Effects of transformations on the variance

In [ ]:
# this gives log2(n + 1)
ntd <- normTransform(dds)
library("vsn")
meanSdPlot(assay(ntd))

In [ ]:
meanSdPlot(assay(vsd))

In [ ]:
meanSdPlot(assay(rld))

# Data quality assessment by sample clustering and visualization

In [ ]:
library("pheatmap")
select <- order(rowMeans(counts(dds,normalized=TRUE)),
                decreasing=TRUE)[1:20]
df <- as.data.frame(colData(dds)[,"condition"])
rownames(df) = colnames(dds)
# ntd
pheatmap(assay(ntd)[select,], cluster_rows=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)

In [ ]:
# vsd
pheatmap(assay(vsd)[select,], cluster_rows=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)

In [ ]:
# rld
pheatmap(assay(rld)[select,], cluster_rows=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)

## Heatmap of the sample-to-sample distances

In [ ]:
sampleDists <- dist(t(assay(vsd)))
library("RColorBrewer")
sampleDistMatrix <- as.matrix(sampleDists)
rownames(sampleDistMatrix) <- paste(vsd$condition, vsd$type, sep="-")
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
pheatmap(sampleDistMatrix,
         clustering_distance_rows=sampleDists,
         clustering_distance_cols=sampleDists,
         col=colors)

## Principal component plot of the samples

In [ ]:
plotPCA(vsd, intgroup=c("condition"))